In [0]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

In [54]:
data = pd.read_csv('/content/Sentiment.csv')

# Keeping only the neccessary columns
data = data[['text','sentiment']]
type(data)

pandas.core.frame.DataFrame

In [0]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

In [31]:
X.shape



(13871, 28)

In [0]:
def createmodel():
  embed_dim=128
  lstm_out=196
  model = Sequential()
  model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(3,activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model
# print(model.summary())

(13871,)

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [67]:
print(data['sentiment'])
print(integer_encoded)

0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object
[1 2 1 ... 2 0 2]


In [92]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
 - 26s - loss: 0.8253 - accuracy: 0.6402
Epoch 2/20
 - 25s - loss: 0.6863 - accuracy: 0.7081
Epoch 3/20
 - 25s - loss: 0.6273 - accuracy: 0.7356
Epoch 4/20
 - 26s - loss: 0.5813 - accuracy: 0.7536
Epoch 5/20
 - 26s - loss: 0.5420 - accuracy: 0.7780
Epoch 6/20
 - 25s - loss: 0.5059 - accuracy: 0.7925
Epoch 7/20
 - 31s - loss: 0.4728 - accuracy: 0.8072
Epoch 8/20
 - 25s - loss: 0.4359 - accuracy: 0.8208
Epoch 9/20
 - 25s - loss: 0.4056 - accuracy: 0.8348
Epoch 10/20
 - 26s - loss: 0.3819 - accuracy: 0.8419
Epoch 11/20
 - 26s - loss: 0.3592 - accuracy: 0.8515
Epoch 12/20
 - 26s - loss: 0.3353 - accuracy: 0.8643
Epoch 13/20
 - 25s - loss: 0.3221 - accuracy: 0.8691
Epoch 14/20
 - 25s - loss: 0.3017 - accuracy: 0.8751
Epoch 15/20
 - 25s - loss: 0.2935 - accuracy: 0.8811
Epoch 16/20
 - 25s - loss: 0.2751 - accuracy: 0.8883
Epoch 17/20
 - 26s - loss: 0.2665 - accuracy: 0.8917
Epoch 18/20
 - 26s - loss: 0.2626 - accuracy: 0.8925
Epoch 19/20
 - 26s - loss: 0.2574 - accuracy: 0.8952
Ep

MODEL SAVED


In [0]:
model.save("model.h5")

EXTRACTING MODEL

In [80]:
from keras.models import load_model
model_save= load_model('/content/model.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


PREDICTING GIVEN RESULTS


In [81]:

import numpy as np
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing']
#vectorizing the tweet by the pre-fitted tokenizer instance
sentence = tokenizer.texts_to_sequences(sentence)
#padding the tweet to have exactly the same shape as `embedding_2` input
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)

sentiment = model_save.predict_classes(sentence,batch_size=1)[0]


if sentiment == 0:
  print("negative")
elif sentiment==1:
     print("neutral")
else:
        print("positve")


positve


2.GRID SEARCH


In [113]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
# model = createmodel()
model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [40, 50, 60]
epochs = [1, 2, 3]
param_grid= dict(batch_size=batch_size, epochs=epochs)
grid  = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result= grid.fit(X_train, y=Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 19s - loss: 0.8547 - accuracy: 0.6318
Epoch 1/1
 - 19s - loss: 0.8521 - accuracy: 0.6321
Epoch 1/1
 - 19s - loss: 0.8511 - accuracy: 0.6340
Epoch 1/1
 - 18s - loss: 0.8486 - accuracy: 0.6328
Epoch 1/1
 - 18s - loss: 0.8477 - accuracy: 0.6355
Epoch 1/2
 - 18s - loss: 0.8436 - accuracy: 0.6329
Epoch 2/2
 - 18s - loss: 0.7006 - accuracy: 0.6991
Epoch 1/2
 - 18s - loss: 0.8491 - accuracy: 0.6349
Epoch 2/2
 - 18s - loss: 0.7043 - accuracy: 0.7027
Epoch 1/2
 - 18s - loss: 0.8558 - accuracy: 0.6340
Epoch 2/2
 - 18s - loss: 0.7038 - accuracy: 0.7035
Epoch 1/2
 - 18s - loss: 0.8517 - accuracy: 0.6362
Epoch 2/2
 - 18s - loss: 0.6961 - accuracy: 0.7017
Epoch 1/2
 - 18s - loss: 0.8550 - accuracy: 0.6343
Epoch 2/2
 - 18s - loss: 0.6869 - accuracy: 0.7059
Epoch 1/3
 - 18s - loss: 0.8516 - accuracy: 0.6334
Epoch 2/3
 - 18s - loss: 0.7030 - accuracy: 0.7008
Epoch 3/3
 - 17s - loss: 0.6265 - accuracy: 0.7341
Epoch 1/3
 - 18s - loss: 0.8442 - accuracy: 0.6349
Epoch 2/3
 - 18s - loss: 0.7029

3. Text Classification On spam.csv

In [0]:
data1 = pd.read_csv('/content/spam.csv',encoding="ISO-8859-1")

# Keeping only the neccessary columns
data1 = data1[['v1','v2']]

In [0]:
data1['v2'] = data1['v2'].apply(lambda x: x.lower())
data1['v2'] = data1['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [0]:
for idx, row in data1.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data1['v2'].values)
X = tokenizer.texts_to_sequences(data1['v2'].values)

X = pad_sequences(X)

In [0]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data1['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [0]:
def createmodel1():
  embed_dim=128
  lstm_out=196
  model = Sequential()
  model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
  model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(2,activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  return model

In [128]:
batch_size = 32
model1 = createmodel1()
model1.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)
score,acc = model1.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model1.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 53s - loss: 0.1708 - accuracy: 0.9387
Epoch 2/5
 - 56s - loss: 0.0414 - accuracy: 0.9863
Epoch 3/5
 - 52s - loss: 0.0207 - accuracy: 0.9928
Epoch 4/5
 - 52s - loss: 0.0109 - accuracy: 0.9965
Epoch 5/5
 - 51s - loss: 0.0066 - accuracy: 0.9981
0.10051059928637093
0.9831430315971375
['loss', 'accuracy']
